# 라이브러리

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pycaret
!pip install scikit-learn
!pip install joblib==1.3
!pip install -q catboost
!pip install --upgrade -q xgboost
!pip install bayesian-optimization==1.4.2
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 8.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from pycaret.regression import setup, compare_models, pull
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import OneHotEncoder
import pycaret
from itertools import permutations
from sklearn.preprocessing import StandardScaler
import itertools
import optuna

from xgboost import XGBRegressor
from sklearn.svm import SVR
from sklearn.linear_model import Ridge, ElasticNet, Lasso, LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from bayes_opt import BayesianOptimization
from sklearn.model_selection import cross_val_score
from lightgbm import LGBMRegressor

# 데이터 불러오기 및 전처리

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/DATATHON/data/@KBO_5개구단/KBO_top5.csv", encoding = 'cp949')

df = df.drop(columns=['선수명', '년도'])

# '타율' 열에서 '-' 값을 가진 행을 제거
df = df[df['타율'] != '-']

# 나머지 열에서 '-' 값을 0으로 변경
df.replace('-', 0, inplace=True)
# NaN 값을 0으로 대체
df.fillna(0, inplace=True)

# 문자열 열을 숫자형으로 변환
df['출루율'] = pd.to_numeric(df['출루율'])
df['장타율'] = pd.to_numeric(df['장타율'])

# 특징과 타겟 설정
features = ['출루율', '장타율', '홈런', '수비 승리 기여도', '득점권타율', '도루허용']
target = '추정득점'

X = df[features]
y = df[target]

# 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)

# 1차 시도


In [ ]:
# StandardSCaler + AUTOML

# 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled = scaler.transform(X_test)

# pycaret을 위한 데이터 준비
X = df[features + [target]]

# pycaret setup
reg = setup(data=X, target=target, session_id=42, normalize=True)

# 여러 모델을 비교하고 상위 7개 모델 선택
top_7_model = compare_models(fold=7, round=3, n_select=7, verbose=True)

,Description,Value
0,Session id,42
1,Target,추정득점
2,Target type,Regression
3,Original data shape,"(678, 7)"
4,Transformed data shape,"(678, 7)"
5,Transformed train set shape,"(474, 7)"
6,Transformed test set shape,"(204, 7)"
7,Numeric features,6
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,6.187,113.973,10.417,0.883,0.565,1.026,2.791
et,Extra Trees Regressor,6.016,113.047,10.428,0.881,0.536,0.886,0.350
rf,Random Forest Regressor,6.246,119.038,10.698,0.873,0.543,0.987,0.617
lightgbm,Light Gradient Boosting Machine,6.743,121.929,10.793,0.872,0.622,1.435,0.943
knn,K Neighbors Regressor,6.646,125.832,10.972,0.869,0.607,1.130,0.103
gbr,Gradient Boosting Regressor,6.766,144.348,11.633,0.851,0.584,1.214,0.139
xgboost,Extreme Gradient Boosting,6.845,160.602,12.231,0.836,0.573,1.122,0.181
llar,Lasso Least Angle Regression,9.152,170.348,12.912,0.824,0.969,5.450,0.071
lasso,Lasso Regression,9.152,170.353,12.912,0.824,0.969,5.448,0.044
ridge,Ridge Regression,9.066,171.569,12.953,0.822,0.974,5.881,0.056


Processing:   0%|          | 0/91 [00:00<?, ?it/s]

# 1차시도 결과

In [ ]:
# 비교 결과를 데이터프레임으로 가져오기
results = pull()

# 상위 7개 모델 추출
top_7_results = results.head(7)

# 상위 7개 모델의 성능 지표 출력
top_7_results

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,6.187,113.973,10.417,0.883,0.565,1.026,2.701
et,Extra Trees Regressor,6.016,113.047,10.428,0.881,0.536,0.886,0.624
rf,Random Forest Regressor,6.246,119.038,10.698,0.873,0.543,0.987,0.530
lightgbm,Light Gradient Boosting Machine,6.743,121.929,10.793,0.872,0.622,1.435,0.649
knn,K Neighbors Regressor,6.646,125.832,10.972,0.869,0.607,1.130,0.050
gbr,Gradient Boosting Regressor,6.766,144.348,11.633,0.851,0.584,1.214,0.436
xgboost,Extreme Gradient Boosting,6.845,160.602,12.231,0.836,0.573,1.122,0.324


# 2차시도

In [ ]:
# 논문에 언급된 7개 모델 (파라미터 값 설정 없이 random_state=42만 설정)

# 모델 목록 (random_state 추가)
models = {
    'XGB': XGBRegressor(),
    'SVR': SVR(),
    'Ridge': Ridge(),
    'RandomForest': RandomForestRegressor(random_state=42),
    'GradientBoosting': GradientBoostingRegressor(),
    'KNN': KNeighborsRegressor(),
    'AdaBoost': AdaBoostRegressor(random_state=42)
}

# 각 모델에 대한 예측 및 결과 출력
results = {}

for name, model in models.items():
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)

    # MSE, RMSE, R2 계산
    mse = mean_squared_error(y_test, predictions)
    rmse = mean_squared_error(y_test, predictions, squared=False)
    mae = mean_absolute_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)

    results[name] = {
        'MSE': mse,
        'RMSE': rmse,
        'MAE': mae,
        'R2 Score': r2
    }

results_df = pd.DataFrame(results).T

# Difference를 기준으로 정렬
results_df_sorted = results_df.sort_values(by='RMSE')

# 결과 출력
for name, result in results_df_sorted.iterrows():
    print(f"Model: {name}")
    print(f"  MSE: {result['MSE']:.2f}")
    print(f"  RMSE: {result['RMSE']:.2f}")
    print(f"  MAE: {result['MAE']}")
    print(f"  R2 Score: {result['R2 Score']:.2f}")
    print()

Model: GradientBoosting
  MSE: 126.91
  RMSE: 11.27
  MAE: 6.234080474028273
  R2 Score: 0.86

Model: RandomForest
  MSE: 135.68
  RMSE: 11.65
  MAE: 6.670533397388092
  R2 Score: 0.85

Model: XGB
  MSE: 160.40
  RMSE: 12.66
  MAE: 7.339164236582378
  R2 Score: 0.83

Model: Ridge
  MSE: 182.09
  RMSE: 13.49
  MAE: 9.231993998461517
  R2 Score: 0.80

Model: KNN
  MSE: 185.50
  RMSE: 13.62
  MAE: 7.872352941176471
  R2 Score: 0.80

Model: AdaBoost
  MSE: 189.42
  RMSE: 13.76
  MAE: 10.250804355791685
  R2 Score: 0.80

Model: SVR
  MSE: 225.85
  RMSE: 15.03
  MAE: 9.383411988866147
  R2 Score: 0.76



# 2차시도 결과

In [ ]:
results_df_sorted

,MSE,RMSE,MAE,R2 Score
GradientBoosting,126.907615,11.265328,6.234080,0.863440
RandomForest,135.678706,11.648120,6.670533,0.854002
XGB,160.397809,12.664826,7.339164,0.827403
Ridge,182.088060,13.494001,9.231994,0.804063
KNN,185.497594,13.619750,7.872353,0.800394
AdaBoost,189.420534,13.763013,10.250804,0.796173
SVR,225.852705,15.028397,9.383412,0.756970


# 3차시도

In [ ]:
# 논문에 언급된 7개 모델 (논문에 쓰인 파라미터값 그대로 사용)

# 모델 목록
models = {
    'XGB': XGBRegressor(objective='reg:squarederror', n_estimators=103, learning_rate=0.03 ,max_depth=2, min_samples_split=23, random_state=0),
    'SVR': SVR(kernel='linear', C=0.05, epsilon=0.10),
    'Ridge': Ridge(alpha = 3.4),
    'RandomForest': RandomForestRegressor(n_estimators=103, max_depth=10, max_features=0.5),
    'GradientBoosting': GradientBoostingRegressor(),
    'KNN': KNeighborsRegressor(n_neighbors = 7),
    'AdaBoost': AdaBoostRegressor(n_estimators=50, random_state=100, learning_rate=2, loss='linear', estimator=Ridge(alpha = 3.4)),

		'lgbm'            	:LGBMRegressor(max_depth =2, min_data_in_leaf = 7),
		'LinearRegression'	:LinearRegression(fit_intercept=True, positive=True),
		'lasso'           	:Lasso(alpha = 0.005),
}

# 각 모델에 대한 예측 및 결과 출력
results = {}

for name, model in models.items():
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)

    # MSE, RMSE, R2 계산
    mse = mean_squared_error(y_test, predictions)
    rmse = mean_squared_error(y_test, predictions, squared=False)
    mae = mean_absolute_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)

    results[name] = {
        'MSE': mse,
        'RMSE': rmse,
        'MAE': mae,
        'R2 Score': r2
    }

results_df = pd.DataFrame(results).T

# Difference를 기준으로 정렬
results_df_sorted = results_df.sort_values(by='RMSE')

# 결과 출력
for name, result in results_df_sorted.iterrows():
    print(f"Model: {name}")
    print(f"  MSE: {result['MSE']:.2f}")
    print(f"  RMSE: {result['RMSE']:.2f}")
    print(f"  MAE: {result['MAE']}")
    print(f"  R2 Score: {result['R2 Score']:.2f}")
    print()

[LightGBM] [Warning] min_data_in_leaf is set=7, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=7
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] min_data_in_leaf is set=7, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=7
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000116 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 537
[LightGBM] [Info] Number of data points in the train set: 542, number of used features: 6
[LightGBM] [Info] Start training from score 25.211624
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits wi

# 3차시도 결과

In [ ]:
results_df_sorted

,MSE,RMSE,MAE,R2 Score
GradientBoosting,126.789776,11.260097,6.227388,0.863567
RandomForest,128.129853,11.319446,6.684208,0.862125
lgbm,130.306868,11.415203,6.503044,0.859782
XGB,137.706194,11.734828,7.557105,0.851820
lasso,182.123283,13.495306,9.306350,0.804025
LinearRegression,182.283759,13.501250,9.328885,0.803852
Ridge,182.582216,13.512299,9.095281,0.803531
KNN,183.443405,13.544128,7.801576,0.802605
SVR,200.631681,14.164451,8.467067,0.784109
AdaBoost,226.211782,15.040338,11.756387,0.756583


# 4차시도

In [ ]:
# 모델별 파라미터의 범위를 지정해 최적의 파라미터 값 찾기

def optimize_model(model_name):
    def objective(trial):
        if model_name == 'XGB':
            params = {
                'objective': 'reg:squarederror',
                'n_estimators': trial.suggest_int('xgb_n_estimators', 50, 200),
                'learning_rate': trial.suggest_float('xgb_learning_rate', 0.01, 0.3),
                'max_depth': trial.suggest_int('xgb_max_depth', 1, 10),
                'random_state': trial.suggest_int('xgb_random_state', 0, 1000)
            }
            model = XGBRegressor(**params)

        elif model_name == 'SVR':
            params = {
                'kernel': 'linear',
                'C': trial.suggest_float('svr_C', 0.01, 10),
                'epsilon': trial.suggest_float('svr_epsilon', 0.01, 1)
            }
            model = SVR(**params)

        elif model_name == 'Ridge':
            params = {
                'alpha': trial.suggest_float('ridge_alpha', 0.1, 10)
            }
            model = Ridge(**params)

        elif model_name == 'RandomForest':
            params = {
                'n_estimators': trial.suggest_int('rf_n_estimators', 50, 200),
                'max_depth': trial.suggest_int('rf_max_depth', 1, 20),
                'max_features': trial.suggest_float('rf_max_features', 0.1, 1.0),
                'random_state': trial.suggest_int('rf_random_state', 0, 1000)
            }
            model = RandomForestRegressor(**params)

        elif model_name == 'GradientBoosting':
            params = {
                'n_estimators': trial.suggest_int('gb_n_estimators', 50, 200),
                'learning_rate': trial.suggest_float('gb_learning_rate', 0.01, 0.3),
                'max_depth': trial.suggest_int('gb_max_depth', 1, 10)
            }
            model = GradientBoostingRegressor(**params)

        elif model_name == 'KNN':
            params = {
                'n_neighbors': trial.suggest_int('knn_n_neighbors', 1, 20)
            }
            model = KNeighborsRegressor(**params)

        elif model_name == 'AdaBoost':
            params = {
                'n_estimators': trial.suggest_int('ada_n_estimators', 50, 200),
                'learning_rate': trial.suggest_float('ada_learning_rate', 0.01, 2),
                'loss': 'linear',
                'estimator': Ridge(alpha=trial.suggest_float('ada_ridge_alpha', 0.1, 10))
            }
            model = AdaBoostRegressor(**params)

        elif model_name == 'LGBM':
            params = {
                'max_depth': trial.suggest_int('lgbm_max_depth', 1, 10),
                'min_data_in_leaf': trial.suggest_int('lgbm_min_data_in_leaf', 1, 100),
                'num_leaves': trial.suggest_int('lgbm_num_leaves', 2, 2**trial.suggest_int('lgbm_max_depth', 1, 10))
            }
            model = LGBMRegressor(**params)

        elif model_name == 'LinearRegression':
            params = {
                'fit_intercept': trial.suggest_categorical('lr_fit_intercept', [True, False]),
                'positive': trial.suggest_categorical('lr_positive', [True, False])
            }
            model = LinearRegression(**params)

        elif model_name == 'Lasso':
            params = {
                'alpha': trial.suggest_float('lasso_alpha', 0.001, 0.1)
            }
            model = Lasso(**params)

        model.fit(X_train, y_train)
        preds = model.predict(X_test)
        rmse = mean_squared_error(y_test, preds, squared=False)

        return rmse

    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=100)

    print(f'Best trial for {model_name}:')
    trial = study.best_trial
    print(f'  Value: {trial.value}')
    print('  Params: ')
    for key, value in trial.params.items():
        print(f'    {key}: {value}')

model_names = ['XGB', 'SVR', 'Ridge', 'RandomForest', 'GradientBoosting', 'KNN', 'AdaBoost', 'LGBM', 'LinearRegression', 'Lasso']

for model_name in model_names:
    optimize_model(model_name)

[I 2024-06-24 05:25:43,153] A new study created in memory with name: no-name-539303e3-3538-447a-8b02-afd8efd27656
[I 2024-06-24 05:25:43,929] Trial 0 finished with value: 11.02063468940394 and parameters: {'xgb_n_estimators': 107, 'xgb_learning_rate': 0.06742009181548961, 'xgb_max_depth': 3, 'xgb_random_state': 6}. Best is trial 0 with value: 11.02063468940394.
[I 2024-06-24 05:25:44,838] Trial 1 finished with value: 11.636748378047193 and parameters: {'xgb_n_estimators': 69, 'xgb_learning_rate': 0.06342327106392794, 'xgb_max_depth': 2, 'xgb_random_state': 198}. Best is trial 0 with value: 11.02063468940394.
[I 2024-06-24 05:25:50,230] Trial 2 finished with value: 12.840236543996598 and parameters: {'xgb_n_estimators': 142, 'xgb_learning_rate': 0.017314565635395112, 'xgb_max_depth': 9, 'xgb_random_state': 290}. Best is trial 0 with value: 11.02063468940394.
[I 2024-06-24 05:25:50,434] Trial 3 finished with value: 12.299617208322672 and parameters: {'xgb_n_estimators': 165, 'xgb_learnin

Best trial for XGB:
  Value: 10.807221342889374
  Params: 
    xgb_n_estimators: 118
    xgb_learning_rate: 0.21350936068660772
    xgb_max_depth: 3
    xgb_random_state: 797


[I 2024-06-24 05:26:07,293] Trial 1 finished with value: 13.861219993645904 and parameters: {'svr_C': 6.875644774128517, 'svr_epsilon': 0.41335664240394515}. Best is trial 0 with value: 13.8390203568768.
[I 2024-06-24 05:26:07,458] Trial 2 finished with value: 13.84709766006383 and parameters: {'svr_C': 7.080125548702532, 'svr_epsilon': 0.8525674107604242}. Best is trial 0 with value: 13.8390203568768.
[I 2024-06-24 05:26:07,681] Trial 3 finished with value: 13.842706771908064 and parameters: {'svr_C': 8.749447712740126, 'svr_epsilon': 0.7807643103065447}. Best is trial 0 with value: 13.8390203568768.
[I 2024-06-24 05:26:07,854] Trial 4 finished with value: 13.837538247115612 and parameters: {'svr_C': 8.149289925072658, 'svr_epsilon': 0.8813351412920226}. Best is trial 4 with value: 13.837538247115612.
[I 2024-06-24 05:26:07,930] Trial 5 finished with value: 13.862662482825973 and parameters: {'svr_C': 2.1028950192717626, 'svr_epsilon': 0.05395868476965916}. Best is trial 4 with value:

Best trial for SVR:
  Value: 13.817588927923056
  Params: 
    svr_C: 9.379394727891961
    svr_epsilon: 0.9905363938672737


[I 2024-06-24 05:26:25,357] Trial 14 finished with value: 13.516579330770666 and parameters: {'ridge_alpha': 3.968467041990685}. Best is trial 13 with value: 13.489004156926763.
[I 2024-06-24 05:26:25,375] Trial 15 finished with value: 13.489002482092136 and parameters: {'ridge_alpha': 0.21227209022737034}. Best is trial 15 with value: 13.489002482092136.
[I 2024-06-24 05:26:25,393] Trial 16 finished with value: 13.501916019830254 and parameters: {'ridge_alpha': 1.9974708687565308}. Best is trial 15 with value: 13.489002482092136.
[I 2024-06-24 05:26:25,412] Trial 17 finished with value: 13.532637488321473 and parameters: {'ridge_alpha': 5.979468796980848}. Best is trial 15 with value: 13.489002482092136.
[I 2024-06-24 05:26:25,429] Trial 18 finished with value: 13.512974702290807 and parameters: {'ridge_alpha': 3.490643524393729}. Best is trial 15 with value: 13.489002482092136.
[I 2024-06-24 05:26:25,450] Trial 19 finished with value: 13.498577863595049 and parameters: {'ridge_alpha'

...

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] min_data_in_leaf is set=19, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=19
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000128 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 537
[LightGBM] [Info] Number of data points in the train set: 542, number of used features: 6
[LightGBM] [Info] Start training from score 25.211624
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM

[I 2024-06-24 05:28:41,880] Trial 6 finished with value: 13.514752128903162 and parameters: {'lr_fit_intercept': False, 'lr_positive': False}. Best is trial 0 with value: 13.490035689337864.
[I 2024-06-24 05:28:41,892] Trial 7 finished with value: 13.528318895920636 and parameters: {'lr_fit_intercept': False, 'lr_positive': True}. Best is trial 0 with value: 13.490035689337864.
[I 2024-06-24 05:28:41,904] Trial 8 finished with value: 13.490035689337864 and parameters: {'lr_fit_intercept': True, 'lr_positive': False}. Best is trial 0 with value: 13.490035689337864.
[I 2024-06-24 05:28:41,918] Trial 9 finished with value: 13.528318895920636 and parameters: {'lr_fit_intercept': False, 'lr_positive': True}. Best is trial 0 with value: 13.490035689337864.
[I 2024-06-24 05:28:41,943] Trial 10 finished with value: 13.514752128903162 and parameters: {'lr_fit_intercept': False, 'lr_positive': False}. Best is trial 0 with value: 13.490035689337864.
[I 2024-06-24 05:28:41,961] Trial 11 finished w

Best trial for LinearRegression:
  Value: 13.490035689337864
  Params: 
    lr_fit_intercept: True
    lr_positive: False


[I 2024-06-24 05:28:43,843] Trial 12 finished with value: 13.485338413843268 and parameters: {'lasso_alpha': 0.05675511750920976}. Best is trial 11 with value: 13.485334033646561.
[I 2024-06-24 05:28:43,868] Trial 13 finished with value: 13.489116428189202 and parameters: {'lasso_alpha': 0.03024666888782624}. Best is trial 11 with value: 13.485334033646561.
[I 2024-06-24 05:28:43,891] Trial 14 finished with value: 13.491809209577886 and parameters: {'lasso_alpha': 0.001866466081056034}. Best is trial 11 with value: 13.485334033646561.
[I 2024-06-24 05:28:43,914] Trial 15 finished with value: 13.485368014484086 and parameters: {'lasso_alpha': 0.06269129886161368}. Best is trial 11 with value: 13.485334033646561.
[I 2024-06-24 05:28:43,947] Trial 16 finished with value: 13.48610050541425 and parameters: {'lasso_alpha': 0.04601241103628548}. Best is trial 11 with value: 13.485334033646561.
[I 2024-06-24 05:28:43,975] Trial 17 finished with value: 13.486413431563664 and parameters: {'lasso

Best trial for Lasso:
  Value: 13.485311713428638
  Params: 
    lasso_alpha: 0.059167902844267324


In [ ]:
# 찾은 파라미터값으로 모델 평가

# 모델 목록
models = {
    'XGB': XGBRegressor(objective='reg:squarederror', n_estimators=139, learning_rate=0.2251380002611682 ,max_depth=3, min_samples_split=23, random_state=0),
    'SVR': SVR(kernel='linear', C=9.991380547355176, epsilon=0.9930014662385143),
    'Ridge': Ridge(alpha = 0.8148545378482661),
    'RandomForest': RandomForestRegressor(n_estimators=101, max_depth=20, max_features=0.2989430702318192),
    'GradientBoosting': GradientBoostingRegressor(n_estimators=115, learning_rate=0.1040277057724564, max_depth=4),
    'KNN': KNeighborsRegressor(n_neighbors = 4),
    'AdaBoost': AdaBoostRegressor(n_estimators=187, random_state=100, learning_rate=2.4191198314864044, loss='linear', estimator=Ridge(alpha = 0.8148545378482661
)),

		'lgbm'            	:LGBMRegressor(max_depth =4, min_data_in_leaf = 5, num_leaves=10),
		'LinearRegression'	:LinearRegression(fit_intercept=True, positive=True),
		'lasso'           	:Lasso(alpha =  0.09996650510883907
),
}


# 각 모델에 대한 예측 및 결과 출력
results = {}

for name, model in models.items():
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)

    # MSE, RMSE, R2 계산
    mse = mean_squared_error(y_test, predictions)
    rmse = mean_squared_error(y_test, predictions, squared=False)
    mae = mean_absolute_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)

    results[name] = {
        'MSE': mse,
        'RMSE': rmse,
        'MAE': mae,
        'R2 Score': r2
    }

results_df = pd.DataFrame(results).T

# Difference를 기준으로 정렬
results_df_sorted = results_df.sort_values(by='RMSE')

# 결과 출력
for name, result in results_df_sorted.iterrows():
    print(f"Model: {name}")
    print(f"  MSE: {result['MSE']:.2f}")
    print(f"  RMSE: {result['RMSE']:.2f}")
    print(f"  MAE: {result['MAE']}")
    print(f"  R2 Score: {result['R2 Score']:.2f}")
    print()

[LightGBM] [Warning] min_data_in_leaf is set=5, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=5
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] min_data_in_leaf is set=5, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=5
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000118 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 537
[LightGBM] [Info] Number of data points in the train set: 542, number of used features: 6
[LightGBM] [Info] Start training from score 25.211624
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits wi

# 4차시도 결과

In [ ]:
results_df_sorted

,MSE,RMSE,MAE,R2 Score
lgbm,117.648746,10.846601,6.086799,0.873403
XGB,127.797747,11.304767,6.583516,0.862482
RandomForest,132.921969,11.529179,7.321231,0.856968
GradientBoosting,134.314967,11.589433,6.769125,0.855469
Ridge,182.047153,13.492485,9.243906,0.804107
lasso,182.058086,13.492890,9.061979,0.804095
LinearRegression,182.283759,13.501250,9.328885,0.803852
KNN,183.554848,13.548242,7.884743,0.802485
SVR,190.958113,13.818759,8.246512,0.794518
AdaBoost,213.877813,14.624562,11.547561,0.769855


# 결론

- 여러 시도 끝에 공통적으로 성능이 좋았던 **RandomForestRegressor**를 최종 모델로 선택
- 이후 RandomForestRegressor의 최적의 파라미터를 찾는 시도를 할 예정